In [34]:
import pandas as pd
import numpy as np

*Time variant indicators are downloaded from WDI for each period. I extracted the zip files with raw data in the 'raw' subfolder of the 'data' folder. Without changing the names of the raw data file, I just added the `year name` at the tail of the original name extracted from WDI, so that I can keep track. Example: `dc261827-d380-414a-b6ff-55a77579a878_Data.csv` was the name for the data for 2000. I changed the name like `dc261827-d380-414a-b6ff-55a77579a878_Data2000.csv`.*

**Preparing the time variant data takes the following steps:**
>1. Pruning the unnecessary rows with formulas from the tail of the data
>2. Renaming the columns
>3. Replacing the '..' values with NaN
>4. Converting all columns to numeric from object type except the 'iso_o' and 'country'
>5. Finding the mean of the variables of that year and imputing that to missing rows
>6. Creating a column with log transformed GDP value of that year
>7. Finding the common countries those are present in the trade data and also in this time variant data
>8. Only keeping the common countries for each year.
>9. Saving the cleaned data as a csv file named like `attributeYEAR.csv`
>10. Finally concatenating all the dataframes on columns and sorting those with country and year for gravity analysis
>11. Saving the concataned data as `attribute2000_2023.csv` in the cleaned folder

In [67]:
at1999 = pd.read_csv('../data/raw/416d66fa-850c-4c72-a6d8-52d9fecefe55_Data.csv')
at2000 = pd.read_csv('../data/raw/b8d427fb-dca4-47c0-b026-c7016418b6ed_Data.csv')
at2001 = pd.read_csv('../data/raw/04b35699-249a-4d66-97a8-bab79c305702_Data.csv')
at2002 = pd.read_csv('../data/raw/12df4673-6147-46d1-99f3-e419b3f30633_Data.csv')
at2003 = pd.read_csv('../data/raw/e9bcb05a-7626-40fb-ba91-0679770c2561_Data.csv')
at2004 = pd.read_csv('../data/raw/4ba7dd03-beef-4117-bcf5-2741582ae3f0_Data.csv')
at2005 = pd.read_csv('../data/raw/8c0c707a-94a0-4070-83d9-a9890aa84763_Data.csv')
at2006 = pd.read_csv('../data/raw/56f4554f-bddc-45c4-9bd7-da4b172d4d3c_Data.csv')
at2007 = pd.read_csv('../data/raw/616da076-f862-4f86-a7b3-7373f5ae84c1_Data.csv')
at2008 = pd.read_csv('../data/raw/2fc7e1df-39de-4796-9eed-6b22fc006594_Data.csv')
at2009 = pd.read_csv('../data/raw/67e77636-af13-402a-9705-edf48957b7e7_Data.csv')
at2010 = pd.read_csv('../data/raw/21dfe8ea-d81a-491b-b5af-5cc76407ebce_Data.csv')
at2011 = pd.read_csv('../data/raw/afedc3eb-c374-45cd-989c-6c3559a87669_Data.csv')
at2012 = pd.read_csv('../data/raw/211c33af-d726-4d73-8184-b4c3a1eb98a8_Data.csv')
at2013 = pd.read_csv('../data/raw/dcfb9366-1163-4249-82b1-10ee87141062_Data.csv')
at2014 = pd.read_csv('../data/raw/208001ea-808a-41e1-a6f5-b26df80309cd_Data.csv')
at2015 = pd.read_csv('../data/raw/edbff4cc-3f9d-4141-81d0-e8217df7ac6e_Data.csv')
at2016 = pd.read_csv('../data/raw/db97317f-3a63-42d8-ab78-5156a215931d_Data.csv')
at2017 = pd.read_csv('../data/raw/358bd016-ff18-4e14-99cf-fff07cb91245_Data.csv')
at2018 = pd.read_csv('../data/raw/67fe077d-c534-487a-9ae1-06e1e4c2cdf7_Data.csv')
at2019 = pd.read_csv('../data/raw/d9e55111-e6c1-4649-9899-5f5de8d3a55e_Data.csv')
at2020 = pd.read_csv('../data/raw/4b321dfd-f890-48d9-92af-5de0091f24ff_Data.csv')
at2021 = pd.read_csv('../data/raw/32827991-5e23-4783-8b4a-dfed21113a1d_Data.csv')
at2022 = pd.read_csv('../data/raw/eaf573f6-9b89-4ad5-bd2a-05fcbd790eaf_Data.csv')
at2023 = pd.read_csv('../data/raw/446d4be1-fb6f-4d27-8275-9045049ee4c6_Data.csv')

In [36]:
total_trade= pd.read_csv("../data/cleaned/total_edgelist.csv")

In [37]:
m = set(at1999['Country Code'].unique()) & set(total_trade['iso_o'].unique())
print(len(m))
print(set(total_trade['iso_o'].unique()) - set(at1999['Country Code'].unique()))

190
{'TWN', 'MSR'}


In [38]:
time_data = {year: globals()['at' + str(year)] for year in range(1999, 2024)}

In [39]:
rename_columns = {
    'Country Code': 'iso_o',
    'Country Name': 'Country',
    'GDP (current US$) [NY.GDP.MKTP.CD]': 'GDP',
    'GDP per capita (current US$) [NY.GDP.PCAP.CD]': 'GDP_pct',
    'Population, total [SP.POP.TOTL]': 'population',
    'Trade (% of GDP) [NE.TRD.GNFS.ZS]': 'trade_gdp_ratio'}

In [40]:
datatype_change = ['GDP', 'GDP_pct', 'population', 'trade_gdp_ratio']

In [41]:
for year, df in time_data.items():
    modified_df = df.rename(columns = rename_columns).iloc[:265, :]
    modified_df[datatype_change] = modified_df[datatype_change].replace('..', np.nan)
    modified_df[datatype_change] = modified_df[datatype_change].apply(lambda x: pd.to_numeric(x, errors='coerce')).round(2)  
    modified_df['GDP'] = modified_df['GDP'].fillna(modified_df['GDP'].mean())
    modified_df['GDP_pct'] = modified_df['GDP_pct'].fillna(modified_df['GDP_pct'].mean())
    modified_df['trade_gdp_ratio'] = modified_df['trade_gdp_ratio'].fillna(modified_df['trade_gdp_ratio'].mean())
    modified_df['log_GDP'] = np.log(modified_df['GDP'])
    modified_df['log_trade_gdp_ratio'] = np.log(modified_df['trade_gdp_ratio'])
    modified_df = modified_df[modified_df['iso_o'].isin(m)]
    modified_df = modified_df.reset_index(drop = True)
    time_data[year] = modified_df

In [42]:
# unpacking dictionary to dataframes 
for year, df in time_data.items():
    globals()[f"at{year}"] = df

In [43]:
#sanity check
at2021.head()

,Time,Time Code,Country,iso_o,GDP,GDP_pct,population,trade_gdp_ratio,log_GDP,log_trade_gdp_ratio
0,2021,YR2021,Afghanistan,AFG,1.426000e+10,356.50,40000412.0,51.410000,23.380724,3.939833
1,2021,YR2021,Albania,ALB,1.803201e+10,6413.28,2811666.0,75.590000,23.615414,4.325324
2,2021,YR2021,Algeria,DZA,1.862312e+11,4160.56,44761099.0,46.840000,25.950255,3.846738
3,2021,YR2021,Andorra,AND,3.324648e+09,42425.70,78364.0,86.283767,21.924629,4.457641
4,2021,YR2021,Angola,AGO,6.650513e+10,1925.87,34532429.0,74.460000,24.920545,4.310262


In [44]:
at2023.isna().sum()

Time                   0
Time Code              0
Country                0
iso_o                  0
GDP                    0
GDP_pct                0
population             0
trade_gdp_ratio        0
log_GDP                0
log_trade_gdp_ratio    0
dtype: int64

In [45]:
for year in range(1999, 2024):
    globals()[f"at{year}"].to_csv(f"../data/cleaned/attribute{year}.csv", encoding = "utf-8", index = False)

### Preparing the RTA data

This dataset is collected from `https://www.ewf.uni-bayreuth.de/en/research/RTA-data/index.html`
### Steps:
1. keeping only the 'rta column and the years after 1999
2. checking for duplicates
3. removing self-loops
4. saving in the cleaned subfolder of the data folder

In [46]:
rta = pd.read_csv("../data/raw/rta_20241028.csv")

In [47]:
rta  = rta.rename(columns = {'exporter': 'iso_o'})
rta  = rta.rename(columns = {'importer': 'iso_d'})
rta = rta[rta['year'] >= 2000]

In [48]:
rta = rta[rta['iso_o'] != rta['iso_d']]

In [49]:
rta['iso_o'].nunique()

280

In [50]:
print(rta.groupby(['iso_o', 'iso_d']).ngroups)

78120


In [51]:
rta.shape[0]

1874880

In [52]:
rta.shape

(1874880, 11)

In [53]:
rta = rta[['iso_o', 'iso_d', 'year', 'rta']].reset_index(drop = True)

In [54]:
rta.head(2)

,iso_o,iso_d,year,rta
0,ABW,AFG,2000,0
1,ABW,AFG,2001,0


In [55]:
#rta.to_csv("../data/cleaned/rta_long.csv", encoding = 'utf-8', index = False)

In [56]:
rta_edgelist = rta.pivot_table(values = 'rta', index = ['iso_o', 'iso_d'], columns = 'year').reset_index()

In [57]:
rta_edgelist[(rta_edgelist['iso_o'] == 'BGD') & (rta_edgelist['iso_d'] == 'IND')] #sanity check 

year,iso_o,iso_d,2000,2001,2002,2003,2004,2005,2006,2007,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
7370,BGD,IND,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [58]:
rta_edgelist.to_csv("../data/cleaned/rta_edgelist.csv", encoding = 'utf-8', index = False)

### Concatenating time variant indicators for all countries in a single data frame

In [59]:
long = pd.concat([at1999, at2000, at2001, at2002, at2003, at2004, at2005, at2006, at2007, at2008,at2009, at2010, at2011, at2012, at2013, at2014, at2015, at2016, at2017, at2018, at2019, at2020, at2021, at2022, at2023], axis = 0) 

In [60]:
print(long['iso_o'].nunique())
print(long['Time'].nunique())
print(long['Time'].unique())

190
25
['1999' '2000' '2001' '2002' '2003' '2004' '2005' '2006' '2007' '2008'
 '2009' '2010' '2011' '2012' '2013' '2014' '2015' '2016' '2017' '2018'
 '2019' '2020' '2021' '2022' '2023']


In [61]:
long.head(2)

,Time,Time Code,Country,iso_o,GDP,GDP_pct,population,trade_gdp_ratio,log_GDP,log_trade_gdp_ratio
0,1999,YR1999,Afghanistan,AFG,8.347859e+11,7566.373695,19887785.0,71.658325,27.450441,4.271909
1,1999,YR1999,Albania,ALB,3.283942e+09,1056.340000,3108778.0,49.900000,21.912310,3.910021


In [62]:
#at2010[at2010['GDP_pct'].isna()]

In [63]:
long_sorted = long[['iso_o', 'Country', 'Time', 'GDP', 'GDP_pct', 'population', 'trade_gdp_ratio', 'log_GDP', 'log_trade_gdp_ratio']].sort_values(['iso_o', 'Country', 'Time']).reset_index(drop= True)

In [64]:
long_sorted.head(3)

,iso_o,Country,Time,GDP,GDP_pct,population,trade_gdp_ratio,log_GDP,log_trade_gdp_ratio
0,ABW,Aruba,1999,1.722905e+09,19216.20,89659.0,164.56,21.267278,5.103275
1,ABW,Aruba,2000,1.873453e+09,20681.02,90588.0,145.07,21.351049,4.977216
2,ABW,Aruba,2001,1.896457e+09,20740.13,91439.0,140.39,21.363253,4.944424


In [65]:
#qq_sorted['first_diff_lnGDP'] = (qq_sorted.groupby(['iso_o', 'Time'])['log_GDP'].transform(first_difference))

In [66]:
long_sorted.to_csv('../data/cleaned/attributes2000_2023.csv', encoding='utf-8', index=False)